In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
json_paths = ['/content/drive/MyDrive/YOLO_Training/T5_Training_Dataset/training_data_Categorical.jsonl',
             '/content/drive/MyDrive/YOLO_Training/T5_Training_Dataset/training_data_DateTime.jsonl',
             '/content/drive/MyDrive/YOLO_Training/T5_Training_Dataset/training_data_Numeric.jsonl',
             '/content/drive/MyDrive/YOLO_Training/T5_Training_Dataset/training_data_OrderedBins.jsonl',
             ]

In [4]:
from datasets import load_dataset, concatenate_datasets
from transformers import T5Tokenizer
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
import torch

In [5]:
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available: True


In [6]:
train_datasets = []
val_datasets = []

In [7]:
# Loop through each file, split, and store separately
for file in json_paths:
    dataset = load_dataset("json", data_files = {"data": file})["data"]

    # Split dataset (90% train, 10% validation)
    split_dataset = dataset.train_test_split(test_size = 0.1, seed = 42)

    train_datasets.append(split_dataset["train"])
    val_datasets.append(split_dataset["test"])

Generating data split: 0 examples [00:00, ? examples/s]

Generating data split: 0 examples [00:00, ? examples/s]

Generating data split: 0 examples [00:00, ? examples/s]

Generating data split: 0 examples [00:00, ? examples/s]

In [8]:
# Merge (union) all train and validation datasets
train_dataset = concatenate_datasets(train_datasets)
val_dataset = concatenate_datasets(val_datasets)

# Print dataset details
print(f"Total training examples: {len(train_dataset)}")
print(f"Total validation examples: {len(val_dataset)}")

Total training examples: 90000
Total validation examples: 10000


In [9]:
# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
max_item = max(train_dataset['target_text'], key = len)
print("Item with maximum length:", max_item)
print("Length of the item:", len(max_item))

tokens = tokenizer.tokenize(max_item)
print("Tokens:", tokens)
print("Number of tokens:", len(tokens))

Item with maximum length: Mapping Value	X Axis Tick
1	01-Jan-2000 04:29:57
2	01-Jan-2000 05:29:57
3	01-Jan-2000 06:29:57
4	01-Jan-2000 07:29:57
5	01-Jan-2000 08:29:57
6	01-Jan-2000 09:29:57
7	01-Jan-2000 10:29:57
8	01-Jan-2000 11:29:57
9	01-Jan-2000 12:29:57
10	01-Jan-2000 13:29:57
11	01-Jan-2000 14:29:57
12	01-Jan-2000 15:29:57
13	01-Jan-2000 16:29:57
14	01-Jan-2000 17:29:57
15	01-Jan-2000 18:29:57
Length of the item: 376
Tokens: ['▁Ma', 'pping', '▁Value', '▁', 'X', '▁A', 'x', 'is', '▁T', 'ick', '▁1', '▁01', '-', 'J', 'an', '-', '2000', '▁04', ':', '29', ':', '57', '▁2', '▁01', '-', 'J', 'an', '-', '2000', '▁', '05', ':', '29', ':', '57', '▁3', '▁01', '-', 'J', 'an', '-', '2000', '▁06', ':', '29', ':', '57', '▁4', '▁01', '-', 'J', 'an', '-', '2000', '▁07', ':', '29', ':', '57', '▁5', '▁01', '-', 'J', 'an', '-', '2000', '▁08', ':', '29', ':', '57', '▁6', '▁01', '-', 'J', 'an', '-', '2000', '▁09', ':', '29', ':', '57', '▁7', '▁01', '-', 'J', 'an', '-', '2000', '▁10', ':', '29', ':', '57

In [11]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],  # Tokenize the input text
        max_length = 256,          # Adjust based on your data length
        padding = "max_length",    # Ensures uniform input size
        truncation = True
    )

    labels = tokenizer(
        examples["target_text"], # Tokenize the expected output
        max_length = 256,
        padding = "max_length",
        truncation = True
    )

    model_inputs["labels"] = labels["input_ids"]  # Set tokenized targets
    return model_inputs

# Apply preprocessing to both training and validation datasets
train_tokenized = train_dataset.map(preprocess_function, batched = True)
val_tokenized = val_dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [12]:
# Define batch size (adjust based on your GPU/CPU memory)
BATCH_SIZE = 32

# Convert Hugging Face datasets to PyTorch DataLoaders
train_dataloader = DataLoader(train_tokenized, batch_size = BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(val_tokenized, batch_size = BATCH_SIZE)

# Print a sample batch
batch = next(iter(train_dataloader))
print("Sample batch keys:", batch.keys())
print("Sample input IDs:", batch["input_ids"])
print("Sample labels:", batch["labels"])


Sample batch keys: dict_keys(['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'])
Sample input IDs: [tensor([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]), tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]), tensor([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8]), tensor([3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586,
        3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586,
        3586, 3586, 3586, 3586, 3586, 3586, 3586, 3586]), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3]), tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]), tensor([71, 71, 71, 71, 71, 

In [13]:
# Load the pretrained T5 model (same architecture as the tokenizer)
model = T5ForConditionalGeneration.from_pretrained("t5-base")

model.to(device)

# Check the model's architecture
print(model)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [14]:
# Define the training arguments (hyperparameters)
training_args = TrainingArguments(
    output_dir = "./results", # Directory to save model checkpoints

    evaluation_strategy = "epoch", # Evaluate after every epoch

    learning_rate = 2e-5, # Learning rate for optimization

    per_device_train_batch_size = 8, # Batch size for training

    per_device_eval_batch_size = 8, # Batch size for evaluation

    num_train_epochs = 3, # Number of epochs to train

    weight_decay = 0.01, # Weight decay for regularization

    save_steps = 10_000, # Save checkpoint every 10,000 steps

    logging_dir = "./logs", # Directory to save logs

    logging_steps = 500, # Log every 500 steps

    fp16 = True, # Enables mixed precision training

    do_train = True, # Start training

    do_eval = True # Evaluate model after training
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# Initialize the Trainer API
trainer = Trainer(
    model = model, # The T5 model we loaded
    args = training_args, # The training configurations
    train_dataset = train_tokenized, # The tokenized training dataset
    eval_dataset = val_tokenized # The tokenized validation dataset
)

In [ ]:
# Start the training process
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: neel-1999 (neel-1999-humber-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


In [ ]:
# Evaluate the Model

eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

In [ ]:
# Save the Fine-Tuned Model
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")